In [1]:
    import requests
    import pandas as pd
    from bs4 import BeautifulSoup
    from io import StringIO
    import time

    def get_day_price(code, max_pages=89):
        base_url = f"https://finance.naver.com/item/sise_day.naver?code={code}"
        headers = {"User-Agent": "Mozilla/5.0"}
        df_total = pd.DataFrame()

        for page in range(1, max_pages + 1):
            url = f"{base_url}&page={page}"
            res = requests.get(url, headers=headers)
            if res.status_code != 200:
                print(f"페이지 {page} 요청 실패: 상태 코드 {res.status_code}")
                break

            soup = BeautifulSoup(res.text, "html.parser")
            table = soup.find("table", class_="type2")
            if table is None:
                print(f"페이지 {page}에서 테이블을 찾을 수 없습니다.")
                break

            df = pd.read_html(StringIO(str(table)), header=0)[0]
            df = df.dropna()

            # 👉 만약 페이지에 데이터가 없는 경우 자동 종료
            if df.empty:
                print(f"\n✅ 데이터 없음: 페이지 {page}에서 종료합니다.")
                break

            # 전일비 처리
            def parse_change(val):
                val = str(val).strip().replace(",", "")
                if val.startswith("상승"):
                    return int(val.replace("상승", "").strip())
                elif val.startswith("하락"):
                    return -int(val.replace("하락", "").strip())
                elif val.startswith("보합"):
                    return 0
                else:
                    return None

            df["전일비"] = df["전일비"].apply(parse_change)

            # 거래량 처리
            df["거래량"] = df["거래량"].astype(str).str.replace(",", "")
            df["거래량"] = df["거래량"].apply(lambda x: int(float(x)))

            # ✅ 출력 – 첫 페이지만 헤더 포함
            if page == 1:
                print(df.to_string(index=False))  # 헤더 포함
            else:
                print(df.to_string(index=False, header=False))  # 헤더 생략

            df_total = pd.concat([df_total, df], ignore_index=True)
            time.sleep(0.5)

        return df_total

    # 종목 코드 및 기업명 설정
    code = "329180"
    company_name = "HD현대중공업"

    # 데이터 수집
    df = get_day_price(code, max_pages=89)

    # 컬럼명 정리
    df.columns = ["날짜", "종가", "전일비", "시가", "고가", "저가", "거래량"]

    # 날짜 형식 변환
    df["날짜"] = pd.to_datetime(df["날짜"])

    # ✅ 날짜 기준으로 중복 제거
    df = df.drop_duplicates(subset="날짜", keep="first")

    # 날짜 내림차순 정렬
    df = df.sort_values("날짜", ascending=False)

    # CSV 파일로 저장
    filename = f"{company_name}.csv"
    df.to_csv(filename, index=False, encoding="utf-8-sig")
    print(f"\n✅ {filename} 파일이 저장되었습니다.")


        날짜       종가   전일비       시가       고가       저가    거래량
2025.04.25 403000.0 27000 398500.0 406000.0 386000.0 715017
2025.04.24 376000.0  9000 367000.0 382500.0 361000.0 636260
2025.04.23 367000.0  3500 366000.0 369500.0 360000.0 213650
2025.04.22 363500.0 11500 349000.0 364000.0 347500.0 213367
2025.04.21 352000.0 -7500 355500.0 358000.0 347000.0 261655
2025.04.18 359500.0 -6500 373000.0 374000.0 355000.0 278251
2025.04.17 366000.0 13000 358500.0 368500.0 356000.0 285662
2025.04.16 353000.0  -500 360000.0 363500.0 350000.0 231501
2025.04.15 353500.0  6000 347000.0 354500.0 342000.0 219655
2025.04.14 347500.0  5000 357500.0 357500.0 344000.0 264186
2025.04.11 342500.0  18500 335000.0 344000.0 332500.0 479330
2025.04.10 324000.0  30500 310000.0 327500.0 305000.0 665708
2025.04.09 293500.0  -1000 301500.0 303000.0 287500.0 405588
2025.04.08 294500.0  19000 285500.0 295500.0 285500.0 288828
2025.04.07 275500.0 -24500 284000.0 289000.0 275500.0 333671
2025.04.04 300000.0   4500 291500.0